In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
from utils import *

# helps in text preprocessing
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer

# helps in LSTM
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Embedding
from tensorflow.keras.callbacks import EarlyStopping


%matplotlib inline

--ip=127.0.0.1


In [2]:
parent_dir = os.path.join(os.path.abspath(os.path.join(os.getcwd(), os.pardir)))

In [3]:

df_train = pd.read_csv(os.path.join(parent_dir, 'Data\PreprocessedData\english_train_preprocess.csv'))
df_dev = pd.read_csv(os.path.join(parent_dir, 'Data\PreprocessedData\english_dev_preprocess.csv'))
df_test = pd.read_csv(os.path.join(parent_dir, 'Data\PreprocessedData\english_test_preprocess.csv'))

In [4]:
df_train = df_train[['preprocessed_text', 'label']]
df_dev = df_dev[['preprocessed_text', 'label']]
df_test = df_test[['preprocessed_text', 'label']]

In [5]:
label_replacement = {
    'Hope_speech': 0,
    'Non_hope_speech': 1,
    'not-English': 2,
}

df_train['label'] = df_train['label'].replace(label_replacement)
df_test['label'] = df_test['label'].replace(label_replacement)
df_dev['label'] = df_dev['label'].replace(label_replacement)

# Drop rows with label 2
df_train = df_train[df_train['label'] != 2]
df_test = df_test[df_test['label'] != 2]
df_dev = df_dev[df_dev['label'] != 2]

In [6]:
X_train, y_train = df_train['preprocessed_text'].to_numpy(), df_train['label'].to_numpy()
X_dev, y_dev = df_dev['preprocessed_text'].to_numpy(), df_dev['label'].to_numpy()
X_test, y_test = df_test['preprocessed_text'].to_numpy(), df_test['label'].to_numpy()

In [7]:
t = Tokenizer()
t.fit_on_texts(X_train)

In [8]:
encoded_train = t.texts_to_sequences(X_train)
encoded_dev = t.texts_to_sequences(X_dev)
encoded_test = t.texts_to_sequences(X_test)

In [9]:
max_length = 8
padded_train = pad_sequences(encoded_train, maxlen=max_length, padding='post')
padded_dev = pad_sequences(encoded_dev, maxlen=max_length, padding='post')
padded_test = pad_sequences(encoded_test, maxlen=max_length, padding='post')

In [10]:
vocab_size = len(t.word_index) + 1

# define the model
model = Sequential()
model.add(Embedding(vocab_size, 24, input_length=max_length))
model.add(LSTM(24, return_sequences=False))
model.add(Dense(1, activation='sigmoid'))

# compile the model
model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy'])

# summarize the model
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 8, 24)             479640    
                                                                 
 lstm (LSTM)                 (None, 24)                4704      
                                                                 
 dense (Dense)               (None, 1)                 25        
                                                                 
Total params: 484,369
Trainable params: 484,369
Non-trainable params: 0
_________________________________________________________________
None


In [11]:
early_stop = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=3)

# fit the model
model.fit(
    x=padded_train,
    y=y_train,
    epochs=100,
    validation_data=(padded_dev, y_dev), verbose=1,
    callbacks=[early_stop]
)

Epoch 1/100
711/711 [==============================] - 5s 5ms/step - loss: 0.2859 - accuracy: 0.9128 - val_loss: 0.2685 - val_accuracy: 0.9050
Epoch 2/100
711/711 [==============================] - 3s 4ms/step - loss: 0.2602 - accuracy: 0.9145 - val_loss: 0.2647 - val_accuracy: 0.9078
Epoch 3/100
711/711 [==============================] - 3s 4ms/step - loss: 0.2540 - accuracy: 0.9151 - val_loss: 0.2622 - val_accuracy: 0.9092
Epoch 4/100
711/711 [==============================] - 3s 4ms/step - loss: 0.2484 - accuracy: 0.9167 - val_loss: 0.2622 - val_accuracy: 0.9102
Epoch 5/100
711/711 [==============================] - 3s 4ms/step - loss: 0.2432 - accuracy: 0.9182 - val_loss: 0.2626 - val_accuracy: 0.9092
Epoch 6/100
711/711 [==============================] - 3s 4ms/step - loss: 0.2372 - accuracy: 0.9195 - val_loss: 0.2621 - val_accuracy: 0.9124
Epoch 7/100
711/711 [==============================] - 3s 4ms/step - loss: 0.2314 - accuracy: 0.9213 - val_loss: 0.2592 - val_accuracy: 0.9088

In [12]:
train_preds = model.predict(padded_train)
dev_preds = model.predict(padded_dev)
test_preds = model.predict(padded_test)

train_preds = np.where(train_preds > 0.5, 1, 0)
dev_preds = np.where(dev_preds > 0.5, 1, 0)
test_preds = np.where(test_preds > 0.5, 1, 0)

train_preds = train_preds.flatten()
dev_preds = dev_preds.flatten()
test_preds = test_preds.flatten()

89/89 [==============================] - 0s 2ms/step


In [13]:
computeAllScores(train_preds, dev_preds, test_preds, False, True)

Accuracy Train:  0.9281442392260334
Accuracy Dev:  0.9091869060190074
Accuracy Test:  0.9099542736545903
Weighted F1 Train:  0.9105945970591142
Weighted F1 Dev:  0.8848876882737164
Weighted F1 Test:  0.8908517109541166
Macro F1 Train:  0.664533444292532
Macro F1 Dev:  0.6029922177639504
Macro F1 Test:  0.6039459134720033
Micro F1 Train:  0.9281442392260334
Micro F1 Dev:  0.9091869060190074
Micro F1 Test:  0.9099542736545903
Weighted Recall Train:  0.9281442392260334
Weighted Recall Dev:  0.9091869060190074
Weighted Recall Test:  0.9099542736545903
Macro Recall Train:  0.6172817756254189
Macro Recall Dev:  0.5750435051404759
Macro Recall Test:  0.5783586579251832
Micro Recall Train:  0.9281442392260334
Micro Recall Dev:  0.9091869060190074
Micro Recall Test:  0.9099542736545903
Confusion Matrix Train: 
[[  474  1488]
 [  146 20632]]
Confusion Matrix Dev: 
[[  44  228]
 [  30 2539]]
Confusion Matrix Test: 
[[  44  206]
 [  50 2543]]
